---
title: "Bike Sharing Prediction API"
author:
  name: "Jose P. Barrantes"
  url: "https://www.linkedin.com/in/jose-barrantes/"
format: html
editor: source
---


## An accesible inference system

In the fast-paced world of data-driven decision-making, one of the problems we face as data scientists/analysts is delivering our data products promptly. Also, distributing them in a manner that doesn't affect our productivity, for example, manually running a Jupyter Notebook or an R script every time someone needs insights, is awfully unproductive.

An idea that is not new, is to deliver the "answers" in an automated, from which the clients can self-serve each time they are in need. After we have a data product, for instance, a predictive model (*machine learning* if you are into buzzwords), we can make available its capabilities to our stakeholders via a micro service.

## An ML model as a microservice

Imagine your (predictive) model being accessed anytime without disturbing your peace or consuming your time; after all, your time is expensive, and you have many tasks. Here is where the idea of microservices comes into action. A microservice is an isolated piece of software that is in charge of a single task (service) and communicates through a well-defined API. You finish your model, pack it in a microservice, and make it available through an API.

This way, the model will always be available, and its consumption won't block other tasks. Also, more products could be potentially developed with the API.

## But what is an API?

An API is an intermediary that enables you to interact with a service without requiring knowledge of how this service works. You just need to know precisely what you want, tell it to the API, and the API will serve you what you requested.

Let's say that after work, you get hungry, so you park your Capital Bikeshare™ bike out of your favorite Mexican restaurant[^1]. You enter the restaurant and sit at a table. Instead of going directly to the kitchen to cook your own meal or asking the chef what you want, you need to place your order through the waiter or waitress. You kindly request your enchiladas, and after a while, they bring your meal to you.

![Allegory of a restaurant. The waiter/waitress is the API; the kitchen is the service (they are dedicated to cooking); you are the client](img/restaurant_api.png)

Notice that you, as the client, are not required to know how to cook enchiladas. Also, you do not interact directly with the people who prepare the food. You get what you want through a waiter (the API).

In summary, an API is the piece of the system you interact with to get a result generated by a service. Producing this result might be complex, but you don't care about that since you only have to deal with the API.

[^1]: This must be how George R. R. Martin feels when foreshadowing a major (traumatic) event in his novels.

## Software engineering for data scientists

Another problem common to data scientists is that research, predictive modeling, and analytics are dirty processes. This is partly due to a lack of training in **software engineering**, which involves **design**, testing, and software maintenance.

We usually have many problems to solve, questions to answer, and limited time at work. This leaves little to no time to apply the best practices to develop our analysis and maintain our code, leaving a trail of technical debt with each delivery. Yet, since most of us write code, we are, in fact, software developers because we develop software. This should be a hard-to-miss hint that software engineering practices must be core in our profession, or at least not ignored.

Here, we will implement some practices core of software engineering like:

- **CI/CD:** Continuous Integration (CI) and Continuous Deployment (CD) with GitHub Actions.
- **Automated tests:** unit, integration, and inference tests with `pytest`.
- **Containerization:** package the application with Docker for consistency across environments (i.e., it should work on every machine with a well-setup Docker).
Dependency Management: For reproducibility, dependencies were specified in a `requirements.txt` file, and development was conducted in an isolated environment (`venv`).
- **Microservices architecture:** The project is a self-contained microservice focused on a single responsibility: predicting bike rentals. It is independently deployable and communicates through HTTP.

We intend to review these concepts in a practical, applied way. This will give us more tools to design and produce more sustainable systems, which we can develop more efficiently through good practices without sacrificing smooth delivery to our clients.

## About the technologies

Through this practical project, we will deal with several technologies, the most important:

- **FastAPI:** a framework to develop APIs with Python. It is easy to learn, and it makes the development of APIs incredibly fast.
- **Uvicorn:** an ASGI (Asynchronous Server Getaway Interface) Server. It handles incoming HTTP requests and sends responses. This lightweight server is appropriate for our microservice.
- **Docker:** a platform to pack our application into isolated "containers" and make them available to be run in several systems. Perfect to build microservices.
- **GitHub Actions:** a CI/CD platform that allows us to automate your development pipelines. We will use them here to automate the software tests and build and push our docker images.
- **pytest:** a Python framework to simplify our software testing.

These tools will make the development and publishing of our microservice a breeze.

## Let's get started

We will start with the mandatory xkcd comic that this kind of articles include so we can continue with the tutorial.

![Exploits of a Mom, by xkcd  Comics. Available at https://xkcd.com/3027/](img/exploits_of_a_mom.png)

I know that this has nothing to do with software engineering or APIs, but little Bobby Tables always cracks me laugh. Now we can continue with the build of our API and some software engineering practices.

## Problem we want to solve

Now, let's focus on the problem we want to solve; this is probably the most crucial step since it is how we **add value** to the business. We are hired to solve analytical problems with evidence (data) as our raw material to produce those solutions.

You probably won't be surprised to learn this project started as a Jupyter Notebook, which I encourage you to [check out here](https://github.com/jospablo777/ml_model_api/blob/main/notebooks/eda_and_toy_model.ipynb). We are using an open data set from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset).

Let's assume we are working for [Capital Bikeshare](https://capitalbikeshare.com/), a company dedicated to providing a self-service bike rental service managed through a mobile app. For the company, it is crucial to predict the peaks of service usage so they can restock or make more available their products when they're more needed. These products can be cloud computing resources during peak hours, bicycle units, human staff, etc. Also it can b convenient to predict the dates and hours of least use for maintenance tasks, like server updates or bike maintenance.

![Capital Bikeshare station outside of the Eastern Market Metro station. Author: Ben Schumin (2010). Licensed under CC BY-SA 3.0](img/Capital_Bikeshare_station_outside_Eastern_Market_Metro.jpg)

Working as a data scientist for a big tech transport company, you will have lots of data and computational resources. On this occasion, you were put in charge of finding a way to predict the days and hours they will have to replenish their stock and increase the server's capacity, investing in more resources only when necessary (saving money). Also, knowing when to schedule the maintenance labor will be valuable as well.

After finishing the data exploration, you know that working days and the entry and exit from work hours are the busiest time frames. Also, climatic conditions can affect the willingness to rent a bike. You are cautious, so you first took the time to understand the phenomenon. Then, you built the model prototype and corroborated that it is possible to forecast the number of rented bikes at an hour-of-the-day granularity level.

Success! You have a model that can help determine the best time to provide more resources or the times for maintenance. Does this mean that you have already finished with the task and can go on to live the good life?[^2] Well, you already know that Jupyter is not a tool that produces clean results and that if you leave it there, your model will have to be run manually each time, which can introduce bugs and errors. This also means that you will have a high and constant influx of people knocking at your door[^3] asking for predictions and insights.

Jupyter is ideal for experimenting and testing but not for delivering results. Hence, you have decided to deploy this first iteration of the model (the prototype) as a microservice that can be consumed via an API. Here, you can assess how the model behaves in the wild and prepare for future iterations. You will start adding value with predictions and valuable information that will help improve your model. 

The engineering team, for example, will quickly adopt the app to know when to conduct maintenance jobs.

[^2]: At the time of writing, I am aware that "no" is the answer since we're still just at the introduction of this tutorial. 
[^3]: It's even worse today, in our modern remote era. The dreaded Microsoft Teams ringtone will constantly haunt you.

## Microservice development

We were assigned a task, so we started by investigating bike rental data and then continued by building a predictive model using the [CatBoost](https://catboost.ai/) algorithm. This model will be the inference engine of our microservice. You can see this part of the project [here](https://github.com/jospablo777/ml_model_api/blob/main/notebooks/eda_and_toy_model.ipynb)[^4]. We saved the model in our project's `predictive_models/` folder.

Filename:  `notebooks/eda_and_toy_model.ipynb`

In [ ]:
#| label: predictive_model
#| include: true
#| eval: false

from catboost import CatBoostRegressor

# --snip--

# Instantiate our model
catboost_model = CatBoostRegressor(
    iterations=1000,       # Boosting iterations
    learning_rate=0.1,
    depth=6,               # Tree depth
    loss_function='RMSE',  # Loss
    verbose=100            # Let it print the learning state every 100 iterations
)

# Train
catboost_model.fit(X_train, y_train, cat_features=categorical_features)

# --snip--

# Save trained model
catboost_model.save_model("../models/catboost_model_19Dec2024.cbm")

[^4]: You probably already noticed that I keep bringing up the link to this notebook. That's because I spent so much time on it, and I was hoping you could look at my precious.
[^5]: Whoops, here is the notebook again. When presenting results to my stakeholders as a habit, I always prepare two decks, one with very high-level results and a second highly technical one with all the math of the statistical/ML models and intricacies of the system design, just in case somebody has a question on the [interesting]{style="text-decoration: line-through;"}  technical part. This is expecting a deep, nerdy discussion about models, gaps, opportunities, and science; to date, the counts for the times I've used this second deck remain at 0. The good news is that it might be a proxy of our stakeholders' trust in us.

## API development

## Containarization

## Automated tests

## Inference

## Microservice up and running


## Whats left?
Model monitoring, automatic training incorporated to the CI/CD, model versioning, high level applications that consume the APIS

# Recommended readings

Docker Deep Dive by Nigel Poulton
Microservice APIs: Using Python, Flask, FastAPI, OpenAPI and more by Jose Haro Peralta